In [1]:
from nltk import sent_tokenize, word_tokenize
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
pos = pd.DataFrame(
    [x.rstrip() for x
     in open("../data/rt-polaritydata/rt-polarity.pos").readlines()],
columns=['review'])
pos['type'] = 'positive'
neg = pd.DataFrame(
    [x.rstrip() for x
     in open("../data/rt-polaritydata/rt-polarity.neg").readlines()],
columns=['review'])
neg['type'] = 'negative'

In [3]:
TRAIN_PROP = 0.8
train_pos = pos.sample(frac=TRAIN_PROP)
train_neg = neg.sample(frac=TRAIN_PROP)
train_reviews = pd.concat([train_pos, train_neg])
test_pos = pos.drop(train_pos.index)
test_neg = neg.drop(train_neg.index)
test_reviews = pd.concat([test_pos, test_neg])

### Ngrams in CountVectorizer just require changing the ngram_range argument:

In [4]:
NGRAM_RANGE = (1, 3)

In [5]:
cv = CountVectorizer(min_df=1, ngram_range=NGRAM_RANGE,
                     tokenizer=word_tokenize)

In [6]:
train_ngrams = cv.fit_transform(train_reviews['review'])

In [7]:
train_ngrams.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

### Before running this next section, can you guess a ballpark for how many features this will give us?

In [ ]:
train_ngrams.shape[1]

## Train our classifier with bag-of-ngrams features

In [8]:
from sklearn.linear_model import LogisticRegressionCV

MAX_ITER = 1000
model = LogisticRegressionCV(max_iter=MAX_ITER)
model.fit(X=train_ngrams, y=train_reviews['type'])

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=1000, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

### Generate test ngrams

In [9]:
test_ngrams = cv.transform(test_reviews['review'])

In [10]:
model.score(X=test_ngrams, y=test_reviews['type'])

0.7626641651031895